# Isolate individual corn cobs

## Author: Miles Roberts

## Last updated: 2021-03-06

## Goals:
* Crop out individual corn cobs from images already cropped by tray

In [1]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
from skimage import exposure #histogram equalization
import colorsys #To convert to rbg to hsv color space
import matplotlib.colors as colors
import os #For getting list of files
from scipy import ndimage #For performing erosion and dilation
import matplotlib.colors as colors
from ipywidgets import interactive,fixed #For interactives

from scipy.signal import argrelextrema #To find local maxima in binary corn images, which will delinate individual cobs

#Function for calculating run lengths in a binary array
#function is from: https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

#Function for smoothing out noisy line plot to emphasize local maxima
#Function from: https://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """
    
    """
    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
    """

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

#Get list of photo names
fileNames = os.listdir(path='.\croppedCornPhotos')
print(fileNames)
os.chdir(path='.\croppedCornPhotos')

for fileName in fileNames:
    im = imread(fileName)
#Thresholding
#Define thresholds for isolating tray in photos
    hmin = 0.11
    hmax = 0.20
    smin = 0.20
    smax = 1.01
    vmin = -1
    vmax = 256

#convert from rgb to hsv color space, pull out matrices
    hsv = colors.rgb_to_hsv(im)
    h = hsv[:,:,0]; #hue matrix
    s = hsv[:,:,1]; #saturation matrix
    v = hsv[:,:,2]; #value matrix (i.e. brightness)
    
#Convert to binary image based on thresholds
# trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
        b_img = (b_img & 
            (s > smin) & (s < smax) & 
            (v > vmin) & (v < vmax));

#Label objects in binary image
    lab, num_features = ndimage.measurements.label(b_img)
#plt.imshow(lab==0, cmap='jet')
#Sum togther rows and columns of binary array to determine which pixels represent the tray (labeled as object 1) 
    a1 = np.sum(lab==0,axis=1)

#Crop off extra tray parts, which will interfere with finding maxima
##Convert binary arrays to logical arrays. Now just need to find longest run of False elements in each array
    al1 = a1 < max(a1)
    
##Calculate run lengths
    runLengths1 = rle(al1)

##Find index of where longest run begins
    runs1 = runLengths1[0]
    positions1 =  runLengths1[1]
    maxRun1 = max(runs1)
    result = np.where(runs1 == maxRun1)
##Calculate where longest run ends
    index = np.asarray(result)
    startRow = positions1[index].tolist()[0][0]
    endRow = startRow + maxRun1

#Crop out end part from array
    a1Cropped = a1[startRow:endRow]

#Smooth out array values, emphasize top 5 local maxima
    a1CroppedSmooth = smooth(a1Cropped,window_len=100,window='hanning')

#Get top local maxima
    maxm = argrelextrema(a1CroppedSmooth, np.greater) 
    a1CroppedSmoothMax = a1CroppedSmooth[maxm]
    a1CroppedSmoothMax.sort()
    a1CroppedSmoothMax
    a1CroppedSmoothMaxTop = a1CroppedSmoothMax[(len(a1CroppedSmoothMax) - 4):len(a1CroppedSmoothMax)]

#Get indices for where top local maxima are, add in begining and ends of entire photo
    maxInd = np.where(np.isin(a1CroppedSmooth, a1CroppedSmoothMaxTop))
    num_rows, num_cols = b_img.shape

    maxInd = np.append(np.array([0]), maxInd, axis=None)
    maxInd = np.append(maxInd, np.array(num_rows), axis=None)
    
#Loop over index pairs, crop photos and output individual corn cobs
    for i in range(len(maxInd) - 1):
        bimgCob = b_img[maxInd[i]:maxInd[i + 1],:]
        imsave("cob" + str(i) + "_"+ "binary" + "_" + fileName, bimgCob * 255)

['cropped_IMG_8248.jpg', 'cropped_IMG_8270.jpg', 'cropped_IMG_8275.jpg', 'cropped_IMG_8277.jpg', 'cropped_IMG_8280.jpg', 'cropped_IMG_8284.jpg', 'cropped_IMG_8291.jpg', 'cropped_IMG_8302.jpg', 'cropped_IMG_8305.jpg', 'cropped_IMG_8312.jpg', 'cropped_IMG_8333.jpg', 'cropped_IMG_8338.jpg', 'cropped_IMG_8339.jpg', 'cropped_IMG_8340.jpg', 'cropped_IMG_8342.jpg', 'cropped_IMG_8343.jpg', 'cropped_IMG_8349.jpg']


Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 